In [5]:
import pandas as pd



In [6]:
#df=pd.read_csv('news_data.csv',engine='python',error_bad_lines=False)
df=pd.read_csv('news_data.csv',encoding='unicode_escape')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We DidnÛªt Even See ComeyÛªs...,Darrell Lucus,House Dem Aide: We DidnÛªt Even See ComeyÛªs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.isnull().sum()

id          0
title      36
author    130
text       78
label     119
dtype: int64

In [9]:
df.shape

(999, 5)

In [10]:
df=df.dropna()

In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We DidnÛªt Even See ComeyÛªs...,Darrell Lucus,House Dem Aide: We DidnÛªt Even See ComeyÛªs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
## get the independent features
X=df.drop('label',axis=1)

In [13]:
y=df['label']

In [14]:
## Check whether dataset is balanced or not
y.value_counts()

0                                                                                                                                                       384
1                                                                                                                                                       287
 by his own account                                                                                                                                       2
 we are one people                                                                                                                                        1
 Vietnam and other countries in the region. These ties are extensive and promising.                                                                       1
                                                                                                                                                       ... 
 for example                                                    

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.9.2'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [18]:
## vocabulary size
voc_size=5000

In [19]:
messages=X.copy()

In [20]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We DidnÛªt Even See ComeyÛªs...,Darrell Lucus,House Dem Aide: We DidnÛªt Even See ComeyÛªs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,BenoÌ¨t Hamon Wins French Socialist PartyÛªs ...,Alissa J. Rubin,"PARIS ÛÓ France chose an idealistic, tradi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,ObamaÛªs Organizing for Action Partners with ...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the ÛÏReal Housewiv...


In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
messages.reset_index(inplace=True)

In [24]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hillari',
 'chuck todd buzzfe donald trump polit favor breitb

In [26]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2341, 156, 1060, 4879, 1576, 368, 796, 3902, 3181, 2109],
 [2481, 3216, 1497, 2507, 3891, 4372, 1189],
 [2906, 4972, 4404, 1997],
 [1963, 2189, 1914, 1278, 3026, 1419],
 [4035, 3891, 4427, 2623, 2094, 3833, 3891, 3277, 4951, 3309],
 [3907,
  168,
  3601,
  1414,
  2641,
  2757,
  3113,
  64,
  2145,
  562,
  2928,
  2948,
  3174,
  1452,
  1189],
 [905, 2681, 1473, 1535, 1987, 399, 4770, 2970, 3813, 1748, 3335],
 [2042, 2939, 1123, 3003, 52, 791, 2757, 3599, 3813, 1748, 3335],
 [4837, 2388, 188, 4259, 1252, 4699, 1478, 4826, 2757, 3080],
 [2847, 538, 2055, 1962, 3033, 4078, 440, 4703],
 [1847, 3346, 936, 3078, 1194, 2757, 4168, 1327, 3596, 3373, 1544],
 [1278, 4294, 1576, 4699, 2757, 52],
 [2987, 2665, 2504, 1464, 3759, 3509, 3982, 3813, 1748, 3335],
 [4115, 1601, 1400, 1267, 780, 3813, 1748, 3335],
 [1871, 3092, 2858, 21, 3812, 3996, 4506, 4464, 3514, 4562],
 [2596, 2203, 3216],
 [2301, 1403, 2348, 2841, 2757, 334, 1305, 1189],
 [3811, 4080, 1497, 295, 4461, 2193, 3797, 773, 3833],


## Embedding Representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 3902, 3181, 2109],
       [   0,    0,    0, ..., 3891, 4372, 1189],
       [   0,    0,    0, ..., 4972, 4404, 1997],
       ...,
       [   0,    0,    0, ..., 3813, 1748, 3335],
       [   0,    0,    0, ..., 3813, 1748, 3335],
       [   0,    0,    0, ...,   87, 3509, 4488]], dtype=int32)

In [28]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10


UnimplementedError: ignored

In [32]:
y_pred=model.predict(X_test)

9/9 [==============================] - 5s 5ms/step


In [33]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [34]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

ValueError: ignored

In [35]:
print(accuracy_score(y_test,y_pred))

0.0
